# Model training lab

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

## Prerequisits

In [1]:
# Imports
import os
import warnings
import tools
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Bidirectional
from matplotlib import pyplot



# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Root CSV files directory
dirname = "./data/"  

# Constant frame count.
frames = 100


#Preparation Stage - Load data and normalize
listfile = os.listdir(dirname)
data = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        data.append((wordname, content))
        
#Split data 60-20-20

features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train, x_val, y_train, y_val = train_test_split(features, labels, test_size=0.40, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.50, random_state=42)

#Enumerate
def printCountDataSets(dataset):
    wortCounter = []
    #Liste mit einmaligen Labels erstellen
    labels = sorted(set(dataset), key=dataset.index)
    #Liste nochmal Alphabetisch sortieren
    labels = sorted(labels)
    for label in labels:
        wortCounter.append(0)
    for row in dataset:
        for i in range(len(labels)):
            if str(labels[i]).startswith(row):
                wortCounter[i] += 1
    for i in range(len(labels)):
        print(labels[i], ': ', wortCounter[i], end =";  ")
    print(' ')        
    
print('Amount Datasets by word total:')
printCountDataSets(labels)
print('')

print('Amount Datasets by word training:')
printCountDataSets(y_train)
print('')

print('Amount Datasets by word validiation:')
printCountDataSets(y_val)
print('')

print('Amount Datasets by word test:')
printCountDataSets(y_test)
print('')


# Display data distribution
print('Distribution of data:')
print("Amount total:", len(labels))
print("Amount training:", len(y_train))
print("Amount validiation:", len(y_val))
print("Amount test:", len(y_test))
print('')

#Tokenize (One Hot)
tokenizer = tools.tokenize(dirname)
print('Tokens:')
print(tokenizer.word_index)
print('')
with open('tokens_json.txt', 'w') as outfile:
    outfile.write(tokenizer.to_json())

encoded_train=tokenizer.texts_to_sequences([y_train])[0]
encoded_val=tokenizer.texts_to_sequences([y_val])[0]
encoded_test=tokenizer.texts_to_sequences([y_test])[0]

y_train = to_categorical(encoded_train)
y_val = to_categorical(encoded_val)
y_test = to_categorical(encoded_test)

print('Categories in OneHot anotation:')
print(y_train)
print('')
# Making numpy arrays
x_train=np.array(x_train)
y_train=np.array(y_train)
x_val=np.array(x_val)
y_val=np.array(y_val)
x_test=np.array(x_test)
y_test=np.array(y_test)

print('Dataset coordinate Values:')
print(x_train)
print('')

import winsound
def finished(num):
    frequency = 2000  # Set Frequency To 2500 Hertz
    duration = 500  # Set Duration To 1000 ms == 1 second
    for i in range(0, num):
        winsound.Beep(frequency, duration)
    

Amount Datasets by word total:
Computer :  57;  Deutschland :  65;  Haben :  68;  Hallo :  57;  Mainz :  65;  Software :  67;  Welt :  66;  du :  66;  ich :  66;  unser :  64;  zeigen :  69;   

Amount Datasets by word training:
Computer :  31;  Deutschland :  30;  Haben :  38;  Hallo :  34;  Mainz :  38;  Software :  46;  Welt :  43;  du :  42;  ich :  37;  unser :  43;  zeigen :  44;   

Amount Datasets by word validiation:
Computer :  13;  Deutschland :  15;  Haben :  11;  Hallo :  11;  Mainz :  10;  Software :  13;  Welt :  14;  du :  13;  ich :  19;  unser :  10;  zeigen :  13;   

Amount Datasets by word test:
Computer :  13;  Deutschland :  20;  Haben :  19;  Hallo :  12;  Mainz :  17;  Software :  8;  Welt :  9;  du :  11;  ich :  10;  unser :  11;  zeigen :  12;   

Distribution of data:
Amount total: 710
Amount training: 426
Amount validiation: 142
Amount test: 142

Tokens:
{'computer': 1, 'deutschland': 2, 'du': 3, 'haben': 4, 'hallo': 5, 'ich': 6, 'mainz': 7, 'software': 8,

## Training Stage
Configure the model and train it.

Metrics:
<div float="right">
    <img src="assets/accuracy.png" width="400"> 
    <img src="assets/precision_recall_formula.png" width="400">
</div>
<img src="assets/precision_recall.png" width="1000">

#### LSTM

In [ ]:

model = Sequential()
model.add(layers.LSTM(256, return_sequences=True,
               input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(layers.LSTM(64, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(layers.LSTM(32))  # return a single vector of dimension 32
#model.add(layers.LSTM(16))  # return a single vector of dimension 32
model.add(layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

### or
#### Bidirectional LSTM

In [ ]:
model = Sequential()
model.add(Bidirectional(layers.LSTM(64, return_sequences=True), input_shape=(100, 86)))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

### or
#### Simple RNN

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(200, 42)))
model.add(Dense(64, activation="softmax")) #softmax, linear 어떤걸 기준으로 하지
model.add(Dense(128, activation="linear")) #softmax, linear 어떤걸 기준으로 하지
model.add(Dense(21))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

### or
#### <span style="color:blue"> Hyperparametertuned LSTM </span>
##### Here it is necessary to install the Keras-Tuner Module by executing:
#####  <span style="color:green"> via Conda:</span>
conda install -c conda-forge keras-tuner
#####  <span style="color:green"> for pip:</span>
pip install keras-tuner


In [14]:
from kerastuner.tuners import RandomSearch
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
from time import time, strftime


starttime= strftime("%Y_%m_%d_%H%M%S")
LOG_DIR = "C:\ML\Optimization_"f"{starttime}" #<-In Windows below Log_dir Path will maybe be too long for Windows to handle, so use a shorter path like this here
#LOG_DIR = "./Optimization_"f"{starttime}" # LOG_DIR holds json files with information and a model of each single trial

def build_model(hp):
    model = Sequential()
    
    model.add(layers.LSTM(hp.Int("LSTM_input", min_value =64, max_value=256,step=64, default=64), #kerastuner will randomly choose a value for nodes between 128 and 256 in steps of 64
                            return_sequences=True,
                            input_shape=(x_train.shape[1], x_train.shape[2])))
    
    for i in range(hp.Int("n_layers" , 1, 3)):    #number of layers ramdom between 1 an 3
        model.add(layers.LSTM(hp.Int(f"LSTM_{i}_units", min_value =64, max_value=256,step=64, default=64),return_sequences=True))
    
    model.add(layers.LSTM(hp.Int(f"LSTM_End", min_value =32, max_value=128,step=32, default=32)))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=hp.Choice('optimizer',values=['Adam','RMSprop','SGD']),
                  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    model.summary()
    print(model.optimizer.get_config()["name"])
    print('')
    return model

###   <span style="color:red">Necesarry only in case of using Nvidia GPU  </span>

In [3]:
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Num GPUs: 1


# Different Keras-Tuner Approaches
### 1 - RandomSearch
Parameter of variables are ranomly used (number of layers, number of nodes) and "best" model is chosen.

In [ ]:
tuner  = RandomSearch(
    build_model,     #Function to use search in
    objective = "val_accuracy",  #Chooses "best model" looking for highest value of val_accuracy
    max_trials = 15,       # Number of different combinations tried Nodes and layers
    executions_per_trial = 1, 
    directory = LOG_DIR,
    project_name='SignLagnuageModelOptimization'
    )

#tuner.search_space_summary()

tuner.search(x=x_train,      #syntax just like in fit
                y= y_train,
            epochs=200,
            batch_size=32,
            validation_data=(x_val,y_val),verbose=2)

print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())

finished(8)

INFO:tensorflow:Reloading Oracle from existing project C:\ML\Optimization_2020_06_29_193027\SignLagnuageModelOptimization\oracle.json
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 64)           38656     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 64)           33024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense (Dense)                (None, 12)                396       
Total params: 84,492
Trainable params: 84,492
Non-trainable params: 0
_________________________________________________________________
Adam

INFO:tensorflow:Reloading Tuner from C:\ML\Optimization_2020_06_29_193027\SignLagnuageModelOptimization\tuner0

Epoch 34/200
426/426 - 0s - loss: 1.3570 - accuracy: 0.4812 - precision: 0.7190 - recall: 0.2042 - val_loss: 1.5432 - val_accuracy: 0.3592 - val_precision: 0.7500 - val_recall: 0.1479
Epoch 35/200
426/426 - 0s - loss: 1.2425 - accuracy: 0.4977 - precision: 0.8083 - recall: 0.2277 - val_loss: 1.5328 - val_accuracy: 0.4014 - val_precision: 0.6500 - val_recall: 0.2746
Epoch 36/200
426/426 - 1s - loss: 1.3627 - accuracy: 0.4812 - precision: 0.7214 - recall: 0.2371 - val_loss: 1.4326 - val_accuracy: 0.4648 - val_precision: 0.5818 - val_recall: 0.2254
Epoch 37/200
426/426 - 0s - loss: 1.2344 - accuracy: 0.5282 - precision: 0.7394 - recall: 0.2465 - val_loss: 1.4287 - val_accuracy: 0.4296 - val_precision: 0.6604 - val_recall: 0.2465
Epoch 38/200
426/426 - 0s - loss: 1.2178 - accuracy: 0.5188 - precision: 0.7401 - recall: 0.3075 - val_loss: 1.5476 - val_accuracy: 0.3521 - val_precision: 0.7174 - val_recall: 0.2324
Epoch 39/200
426/426 - 0s - loss: 1.2336 - accuracy: 0.5070 - precision: 0.7203 

Epoch 79/200
426/426 - 0s - loss: 0.6900 - accuracy: 0.7277 - precision: 0.7726 - recall: 0.6620 - val_loss: 1.0320 - val_accuracy: 0.6268 - val_precision: 0.6829 - val_recall: 0.5915
Epoch 80/200
426/426 - 0s - loss: 0.5904 - accuracy: 0.7746 - precision: 0.8117 - recall: 0.7183 - val_loss: 1.0669 - val_accuracy: 0.6479 - val_precision: 0.6891 - val_recall: 0.5775
Epoch 81/200
426/426 - 0s - loss: 0.6930 - accuracy: 0.7136 - precision: 0.7618 - recall: 0.6831 - val_loss: 0.9102 - val_accuracy: 0.6831 - val_precision: 0.7154 - val_recall: 0.6197
Epoch 82/200
426/426 - 0s - loss: 0.7329 - accuracy: 0.7160 - precision: 0.7647 - recall: 0.6714 - val_loss: 1.2638 - val_accuracy: 0.6268 - val_precision: 0.6638 - val_recall: 0.5423
Epoch 83/200
426/426 - 0s - loss: 0.5991 - accuracy: 0.7653 - precision: 0.8053 - recall: 0.7089 - val_loss: 0.9662 - val_accuracy: 0.6831 - val_precision: 0.7479 - val_recall: 0.6268
Epoch 84/200
426/426 - 0s - loss: 0.5477 - accuracy: 0.7770 - precision: 0.8081 

Epoch 124/200
426/426 - 1s - loss: 0.2583 - accuracy: 0.9131 - precision: 0.9234 - recall: 0.9061 - val_loss: 0.8128 - val_accuracy: 0.7887 - val_precision: 0.8162 - val_recall: 0.7817
Epoch 125/200
426/426 - 0s - loss: 0.3810 - accuracy: 0.8592 - precision: 0.8717 - recall: 0.8451 - val_loss: 1.1234 - val_accuracy: 0.7042 - val_precision: 0.7405 - val_recall: 0.6831
Epoch 126/200
426/426 - 0s - loss: 0.2881 - accuracy: 0.9108 - precision: 0.9185 - recall: 0.8991 - val_loss: 1.2345 - val_accuracy: 0.7113 - val_precision: 0.7259 - val_recall: 0.6901
Epoch 127/200
426/426 - 0s - loss: 0.3778 - accuracy: 0.8662 - precision: 0.8705 - recall: 0.8521 - val_loss: 1.0646 - val_accuracy: 0.7324 - val_precision: 0.7305 - val_recall: 0.7254
Epoch 128/200
426/426 - 0s - loss: 0.2316 - accuracy: 0.9272 - precision: 0.9267 - recall: 0.9202 - val_loss: 1.2960 - val_accuracy: 0.6690 - val_precision: 0.6765 - val_recall: 0.6479
Epoch 129/200
426/426 - 0s - loss: 0.3766 - accuracy: 0.8873 - precision: 0

Epoch 169/200
426/426 - 0s - loss: 0.1726 - accuracy: 0.9413 - precision: 0.9480 - recall: 0.9413 - val_loss: 1.2788 - val_accuracy: 0.6761 - val_precision: 0.6835 - val_recall: 0.6690
Epoch 170/200
426/426 - 1s - loss: 0.3545 - accuracy: 0.8685 - precision: 0.8908 - recall: 0.8615 - val_loss: 0.9034 - val_accuracy: 0.8028 - val_precision: 0.8296 - val_recall: 0.7887
Epoch 171/200
426/426 - 0s - loss: 0.1539 - accuracy: 0.9507 - precision: 0.9551 - recall: 0.9484 - val_loss: 1.0920 - val_accuracy: 0.7606 - val_precision: 0.7681 - val_recall: 0.7465
Epoch 172/200
426/426 - 0s - loss: 0.0970 - accuracy: 0.9812 - precision: 0.9812 - recall: 0.9789 - val_loss: 1.4293 - val_accuracy: 0.7113 - val_precision: 0.7163 - val_recall: 0.7113
Epoch 173/200
426/426 - 0s - loss: 0.2263 - accuracy: 0.9319 - precision: 0.9362 - recall: 0.9296 - val_loss: 0.9398 - val_accuracy: 0.7254 - val_precision: 0.7313 - val_recall: 0.6901
Epoch 174/200
426/426 - 0s - loss: 0.2409 - accuracy: 0.9413 - precision: 0

...

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 128)          110080    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 128)          131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 192)          246528    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 192)          295680    
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                28800     
_________________________________________________________________
dense (Dense)                (None, 12)                396       
Total params: 813,068
Trainable params: 813,068
Non-trainable params: 0
__________________________________________________

Epoch 38/200
426/426 - 1s - loss: 1.6134 - accuracy: 0.3873 - precision: 0.6032 - recall: 0.0892 - val_loss: 1.5174 - val_accuracy: 0.3873 - val_precision: 0.8696 - val_recall: 0.1408
Epoch 39/200
426/426 - 1s - loss: 1.5285 - accuracy: 0.3662 - precision: 0.7500 - recall: 0.0986 - val_loss: 1.6872 - val_accuracy: 0.3521 - val_precision: 0.8148 - val_recall: 0.1549
Epoch 40/200
426/426 - 1s - loss: 1.7094 - accuracy: 0.3380 - precision: 0.6800 - recall: 0.0798 - val_loss: 2.1153 - val_accuracy: 0.2324 - val_precision: 0.7500 - val_recall: 0.0845
Epoch 41/200
426/426 - 1s - loss: 2.0340 - accuracy: 0.2371 - precision: 0.4902 - recall: 0.0587 - val_loss: 1.8095 - val_accuracy: 0.2746 - val_precision: 0.5625 - val_recall: 0.0634
Epoch 42/200
426/426 - 1s - loss: 1.8323 - accuracy: 0.2793 - precision: 0.3077 - recall: 0.0094 - val_loss: 1.7562 - val_accuracy: 0.3028 - val_precision: 0.8750 - val_recall: 0.0493
Epoch 43/200
426/426 - 1s - loss: 1.8132 - accuracy: 0.2746 - precision: 0.4783 

Epoch 83/200
426/426 - 1s - loss: 1.0392 - accuracy: 0.6315 - precision: 0.7122 - recall: 0.4531 - val_loss: 1.2789 - val_accuracy: 0.4789 - val_precision: 0.5823 - val_recall: 0.3239
Epoch 84/200
426/426 - 1s - loss: 1.1148 - accuracy: 0.5493 - precision: 0.6930 - recall: 0.3709 - val_loss: 1.3088 - val_accuracy: 0.4507 - val_precision: 0.5732 - val_recall: 0.3310
Epoch 85/200
426/426 - 1s - loss: 1.2444 - accuracy: 0.5587 - precision: 0.6765 - recall: 0.3779 - val_loss: 1.1968 - val_accuracy: 0.5915 - val_precision: 0.6709 - val_recall: 0.3732
Epoch 86/200
426/426 - 1s - loss: 1.1062 - accuracy: 0.6315 - precision: 0.7031 - recall: 0.4836 - val_loss: 1.2178 - val_accuracy: 0.5493 - val_precision: 0.6292 - val_recall: 0.3944
Epoch 87/200
426/426 - 1s - loss: 0.9867 - accuracy: 0.6526 - precision: 0.7357 - recall: 0.4836 - val_loss: 1.4081 - val_accuracy: 0.5141 - val_precision: 0.5673 - val_recall: 0.4155
Epoch 88/200
426/426 - 1s - loss: 0.9283 - accuracy: 0.6690 - precision: 0.7706 

Epoch 128/200
426/426 - 1s - loss: 0.6139 - accuracy: 0.7746 - precision: 0.8205 - recall: 0.7512 - val_loss: 0.9216 - val_accuracy: 0.6831 - val_precision: 0.7308 - val_recall: 0.6690
Epoch 129/200
426/426 - 1s - loss: 0.5516 - accuracy: 0.7723 - precision: 0.8229 - recall: 0.7418 - val_loss: 0.9621 - val_accuracy: 0.7113 - val_precision: 0.7308 - val_recall: 0.6690
Epoch 130/200
426/426 - 1s - loss: 0.5873 - accuracy: 0.7864 - precision: 0.8293 - recall: 0.7300 - val_loss: 1.2492 - val_accuracy: 0.5986 - val_precision: 0.6281 - val_recall: 0.5352
Epoch 131/200
426/426 - 1s - loss: 0.6168 - accuracy: 0.7958 - precision: 0.8391 - recall: 0.7347 - val_loss: 1.0296 - val_accuracy: 0.6831 - val_precision: 0.6889 - val_recall: 0.6549
Epoch 132/200
426/426 - 1s - loss: 0.4709 - accuracy: 0.8427 - precision: 0.8670 - recall: 0.7958 - val_loss: 1.0565 - val_accuracy: 0.6901 - val_precision: 0.7209 - val_recall: 0.6549
Epoch 133/200
426/426 - 1s - loss: 0.5128 - accuracy: 0.8099 - precision: 0

Epoch 173/200
426/426 - 1s - loss: 0.3770 - accuracy: 0.8803 - precision: 0.8995 - recall: 0.8615 - val_loss: 1.0677 - val_accuracy: 0.6901 - val_precision: 0.7287 - val_recall: 0.6620
Epoch 174/200
426/426 - 1s - loss: 0.3573 - accuracy: 0.8897 - precision: 0.8986 - recall: 0.8732 - val_loss: 1.3971 - val_accuracy: 0.6056 - val_precision: 0.6194 - val_recall: 0.5845
Epoch 175/200
426/426 - 1s - loss: 0.3794 - accuracy: 0.8873 - precision: 0.9037 - recall: 0.8592 - val_loss: 1.0689 - val_accuracy: 0.7042 - val_precision: 0.7293 - val_recall: 0.6831
Epoch 176/200
426/426 - 1s - loss: 0.3931 - accuracy: 0.8756 - precision: 0.8927 - recall: 0.8592 - val_loss: 1.0350 - val_accuracy: 0.6620 - val_precision: 0.7176 - val_recall: 0.6620
Epoch 177/200
426/426 - 1s - loss: 0.2728 - accuracy: 0.9225 - precision: 0.9333 - recall: 0.9202 - val_loss: 0.9921 - val_accuracy: 0.7254 - val_precision: 0.7481 - val_recall: 0.7113
Epoch 178/200
426/426 - 1s - loss: 0.5197 - accuracy: 0.8239 - precision: 0

### 2 - Hyperband
Variation of RandomSearch http://jmlr.org/papers/volume18/16-558/16-558.pdf

In [ ]:
tuner  = Hyperband(
    build_model,
    objective = "val_accuracy",
    hyperband_iterations=2,
    max_epochs=150,
    directory = LOG_DIR,
    project_name='SignLagnuageModelOptimization'
    )

#tuner.search_space_summary()

tuner.search(x=x_train, 
            y= y_train,
            batch_size=32,
            validation_data=(x_val,y_val))

print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())

finished(8)

In [11]:
#Laut Randomsearch bestes Model am 23.06.2020

model = Sequential()
model.add(layers.LSTM(128, return_sequences=True,
               input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(layers.LSTM(64, return_sequences=True)) 
model.add(layers.LSTM(96))  
model.add(layers.Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
model.summary()

#history=model.fit(x_train,y_train,epochs=170,validation_data=(x_val,y_val),shuffle=False)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 100, 128)          110080    
_________________________________________________________________
lstm_12 (LSTM)               (None, 100, 64)           49408     
_________________________________________________________________
lstm_13 (LSTM)               (None, 96)                61824     
_________________________________________________________________
dense_3 (Dense)              (None, 12)                1164      
Total params: 222,476
Trainable params: 222,476
Non-trainable params: 0
_________________________________________________________________


In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

## Export tuner object into pickle file
so it can be used in other scripts

In [ ]:
import pickle

with open(f"tuner_"f"{starttime}.pkl", "wb") as f:
    pickle.dump(tuner, f)
    

## Get best Trial from Tuner Object

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
bestmodel= tuner.hypermodel.build(best_hp)

bestmodel.summary()


In [12]:
#tmp_chekpoints= "tmp\epoch{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}.hdf5"
tmp_chekpoints= "C:\\ML\\checkpoints\\tmp\\epoch{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}.hdf5"

#csv_log = tf.keras.callbacks.CSVLogger("log.csv", separator=',', append=False)
csv_log = tf.keras.callbacks.CSVLogger("C:\ML\logs\log.csv", separator=',', append=False)

#tb = tf.keras.callbacks.TensorBoard(log_dir='logs', histogram_freq=1, write_graph=False, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=1, embeddings_metadata=None)
tb = tf.keras.callbacks.TensorBoard(log_dir='C:\ML\logs', histogram_freq=1, write_graph=False, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=1, embeddings_metadata=None)
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=20, verbose=0, mode='max', baseline=None, restore_best_weights=True)
chk= tf.keras.callbacks.ModelCheckpoint(tmp_chekpoints, monitor='val_accuracy', verbose=0, save_best_only=False, save_weights_only=False, mode='max', save_freq='epoch')


history = model.fit(x_train,y_train,epochs=200,batch_size=32, validation_data=(x_val,y_val),shuffle=False, verbose=2, callbacks=[csv_log, chk])

Train on 426 samples, validate on 142 samples
Epoch 1/200
426/426 - 6s - loss: 2.3897 - accuracy: 0.1268 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: 2.1656 - val_accuracy: 0.2113 - val_precision_3: 1.0000 - val_recall_3: 0.0141
Epoch 2/200
426/426 - 0s - loss: 2.0374 - accuracy: 0.2113 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: 1.9030 - val_accuracy: 0.2254 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00
Epoch 3/200
426/426 - 0s - loss: 1.9257 - accuracy: 0.2441 - precision_3: 1.0000 - recall_3: 0.0047 - val_loss: 1.8837 - val_accuracy: 0.1972 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00
Epoch 4/200
426/426 - 0s - loss: 1.8951 - accuracy: 0.2277 - precision_3: 0.3333 - recall_3: 0.0094 - val_loss: 1.8002 - val_accuracy: 0.2535 - val_precision_3: 0.5000 - val_recall_3: 0.0563
Epoch 5/200
426/426 - 0s - loss: 1.8575 - accuracy: 0.2394 - precision_3: 0.3571 - recall_3: 0.0117 - val_loss: 1.8280 - val_accuracy: 0.2183 - val_precision_

Epoch 44/200
426/426 - 0s - loss: 1.0733 - accuracy: 0.5892 - precision_3: 0.7725 - recall_3: 0.3427 - val_loss: 1.4230 - val_accuracy: 0.4437 - val_precision_3: 0.5965 - val_recall_3: 0.2394
Epoch 45/200
426/426 - 0s - loss: 1.1044 - accuracy: 0.5728 - precision_3: 0.7463 - recall_3: 0.3592 - val_loss: 1.3158 - val_accuracy: 0.5141 - val_precision_3: 0.6769 - val_recall_3: 0.3099
Epoch 46/200
426/426 - 0s - loss: 1.0271 - accuracy: 0.6033 - precision_3: 0.7692 - recall_3: 0.3991 - val_loss: 1.3047 - val_accuracy: 0.4718 - val_precision_3: 0.6216 - val_recall_3: 0.3239
Epoch 47/200
426/426 - 0s - loss: 0.9767 - accuracy: 0.6080 - precision_3: 0.7902 - recall_3: 0.4155 - val_loss: 1.2557 - val_accuracy: 0.5070 - val_precision_3: 0.6512 - val_recall_3: 0.3944
Epoch 48/200
426/426 - 0s - loss: 0.9794 - accuracy: 0.6268 - precision_3: 0.7572 - recall_3: 0.4906 - val_loss: 1.2791 - val_accuracy: 0.5000 - val_precision_3: 0.5941 - val_recall_3: 0.4225
Epoch 49/200
426/426 - 0s - loss: 0.9386

Epoch 87/200
426/426 - 0s - loss: 0.4675 - accuracy: 0.8310 - precision_3: 0.8575 - recall_3: 0.7629 - val_loss: 0.9686 - val_accuracy: 0.6761 - val_precision_3: 0.7231 - val_recall_3: 0.6620
Epoch 88/200
426/426 - 0s - loss: 0.4266 - accuracy: 0.8404 - precision_3: 0.8692 - recall_3: 0.7958 - val_loss: 1.0007 - val_accuracy: 0.6408 - val_precision_3: 0.6855 - val_recall_3: 0.5986
Epoch 89/200
426/426 - 0s - loss: 0.4682 - accuracy: 0.7981 - precision_3: 0.8458 - recall_3: 0.7723 - val_loss: 0.9906 - val_accuracy: 0.6479 - val_precision_3: 0.6929 - val_recall_3: 0.6197
Epoch 90/200
426/426 - 0s - loss: 0.4726 - accuracy: 0.8146 - precision_3: 0.8632 - recall_3: 0.7700 - val_loss: 1.1369 - val_accuracy: 0.6479 - val_precision_3: 0.6692 - val_recall_3: 0.6127
Epoch 91/200
426/426 - 0s - loss: 0.5177 - accuracy: 0.7864 - precision_3: 0.8295 - recall_3: 0.7653 - val_loss: 1.1175 - val_accuracy: 0.6408 - val_precision_3: 0.6692 - val_recall_3: 0.6127
Epoch 92/200
426/426 - 0s - loss: 0.4687

Epoch 130/200
426/426 - 0s - loss: 0.2883 - accuracy: 0.8826 - precision_3: 0.9050 - recall_3: 0.8498 - val_loss: 1.1104 - val_accuracy: 0.6620 - val_precision_3: 0.7154 - val_recall_3: 0.6549
Epoch 131/200
426/426 - 0s - loss: 0.2558 - accuracy: 0.8991 - precision_3: 0.9122 - recall_3: 0.8779 - val_loss: 1.0443 - val_accuracy: 0.6972 - val_precision_3: 0.7218 - val_recall_3: 0.6761
Epoch 132/200
426/426 - 0s - loss: 0.2297 - accuracy: 0.9108 - precision_3: 0.9227 - recall_3: 0.8967 - val_loss: 1.1251 - val_accuracy: 0.6972 - val_precision_3: 0.7059 - val_recall_3: 0.6761
Epoch 133/200
426/426 - 0s - loss: 0.1997 - accuracy: 0.9131 - precision_3: 0.9306 - recall_3: 0.9131 - val_loss: 0.9859 - val_accuracy: 0.7042 - val_precision_3: 0.7259 - val_recall_3: 0.6901
Epoch 134/200
426/426 - 0s - loss: 0.2180 - accuracy: 0.9202 - precision_3: 0.9353 - recall_3: 0.9155 - val_loss: 1.1168 - val_accuracy: 0.6831 - val_precision_3: 0.7132 - val_recall_3: 0.6831
Epoch 135/200
426/426 - 0s - loss: 

Epoch 173/200
426/426 - 0s - loss: 0.2579 - accuracy: 0.9202 - precision_3: 0.9238 - recall_3: 0.9108 - val_loss: 1.1587 - val_accuracy: 0.7394 - val_precision_3: 0.7574 - val_recall_3: 0.7254
Epoch 174/200
426/426 - 0s - loss: 0.2263 - accuracy: 0.9225 - precision_3: 0.9286 - recall_3: 0.9155 - val_loss: 1.0404 - val_accuracy: 0.7606 - val_precision_3: 0.7660 - val_recall_3: 0.7606
Epoch 175/200
426/426 - 0s - loss: 0.1759 - accuracy: 0.9343 - precision_3: 0.9409 - recall_3: 0.9343 - val_loss: 1.0924 - val_accuracy: 0.7535 - val_precision_3: 0.7571 - val_recall_3: 0.7465
Epoch 176/200
426/426 - 0s - loss: 0.2259 - accuracy: 0.9178 - precision_3: 0.9236 - recall_3: 0.9085 - val_loss: 1.0800 - val_accuracy: 0.7113 - val_precision_3: 0.7206 - val_recall_3: 0.6901
Epoch 177/200
426/426 - 0s - loss: 0.3140 - accuracy: 0.8944 - precision_3: 0.9038 - recall_3: 0.8826 - val_loss: 1.3396 - val_accuracy: 0.6972 - val_precision_3: 0.7197 - val_recall_3: 0.6690
Epoch 178/200
426/426 - 0s - loss: 

### Diagnostic Plots

The training history of your LSTM models can be used to diagnose the behavior of your model.

You can plot the performance of your model using the Matplotlib library. For example, you can plot training loss vs test loss as follows:

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.savefig("C:/ML/loss"f"{starttime}.png")
pyplot.show()

pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='lower right')
pyplot.savefig("C:/ML/accuracy_"f"{starttime}.png")
pyplot.show()

#### Underfit Example
Running this example produces a plot of train and validation loss showing the characteristic of an underfit model. In this case, performance may be improved by increasing the number of training epochs.


<img src="assets/Diagnostic-Line-Plot-Showing-an-Underfit-Model.png" width="400">


Running this example shows the characteristic of an underfit model that appears under-provisioned.
In this case, performance may be improved by increasing the capacity of the model, such as the number of memory cells in a hidden layer or number of hidden layers.

<img src="assets/Diagnostic-Line-Plot-Showing-an-Underfit-Model-via-Status.png" width="400">

#### Good Fit Example
Running the example creates a line plot showing the train and validation loss meeting.
Ideally, we would like to see model performance like this if possible, although this may not be possible on challenging problems with a lot of data.

<img src="assets/Diagnostic-Line-Plot-Showing-a-Good-Fit-for-a-Model.png" width="400">

#### Overfit Example
Running this example creates a plot showing the characteristic inflection point in validation loss of an overfit model.
This may be a sign of too many training epochs.
In this case, the model training could be stopped at the inflection point. Alternately, the number of training examples could be increased.

<img src="assets/Diagnostic-Line-Plot-Showing-an-Overfit-Model.png" width="400">

### Evaluate

In [ ]:
#model = tf.keras.models.load_model('./tmp/epoch49-0.90-0.39.hdf5')


#bestmodel.evaluate(x=x_test, y=y_test, verbose=2)
model.evaluate(x=x_test, y=y_test, verbose=2)


### Save model

In [ ]:
bestmodel.save("sign_lang_recognition_tuned.h5")